In [10]:
import codecs
import io, json, codecs
import os
from bs4 import BeautifulSoup as bs
import fix

In [13]:
def oai2jsonld(jsonld_src,issn,journal_name,oai_file):
    soup=bs(xml_content,"lxml").findAll('metadata')
    global articles; articles==[]
    global e
    for s in soup:
        creators=[] 
        article={}
        article['e:journal']=journal_name
        article['e:issn']=issn
        article['e:author']=[]
        article['e:keyword']=[]
        properties=s.findChild().findAll()
        for p in properties:
            key= p.name
            j=p.text.strip()
            if key =='dc:title':article['e:title']=j
            elif key=='dc:date':article['e:year']=int(j[0:4])
            elif key=='dc:identifier':
                if 'vol' in j:
                    if 'iss' in j:
                        issue=j.split('iss')[1].split('/')[0]
                        article['e:issue']=issue                        
                    volume=j.split('vol')[1].split('/')[0]
                    article['e:volume']=volume
                    article['e:url']=j
                    article['e:doi']=j.replace('http://dx.doi.org/','').replace('http://hdl.handle.net/','')
            elif key=='dc:description':article['e:abstract']=j
            elif key=='dc:creator':
                if ',' in j:
                    j=' '.join(reversed([part.strip() for part in j.split(',')]))
                creator=j
                creators.append(creator)
            elif key=='dc:subject': article['e:keyword'].append(fix.fix_keyword(j))
        article['e:author']=creators

        if fix.validate(article)==True:
            articles.append(article)
        else:
            e=e+1
    jd=unicode(json.dumps(articles, ensure_ascii=False))
    jsonld_file=jsonld_src+issn+'_'+oai_file.replace('.xml','.jsonld').replace('/','_')
    with io.open(jsonld_file, 'w') as f:
        f.write(jd)

        return (jsonld_file)

In [14]:
oai_folder='oai/'
jsonld_src='jsonld_src/'

with open('../OpenScience/subject_journals.json') as data_file:    
    subject_journals = json.load(data_file)

journals=[]
for subject in subject_journals:
    journals=journals+(subject['journals'])
print len(journals)
    
for journal in journals:
    issn=journal['issn']
    source=journal['jname']
    journal_name=journal['journal']
    data=journal['data']
    for type_pub in data:   
        file_type=type_pub['type']
        publisher=type_pub['publisher']
        if file_type=='oai':
            print publisher,source,journal_name,
            i=0
            e=0
            for file_name in os.listdir(oai_folder+publisher+'/'+source):
                if file_name[-4:]=='.xml':
                    oai_file=publisher+'/'+source+'/'+file_name
                    articles=[]
                    with codecs.open(oai_folder+oai_file) as myfile:
                        xml_content=myfile.read().replace('\n', '')
                    jsonld_file= oai2jsonld(jsonld_src,issn,journal_name,oai_file)
                i=i+1
            print 'fixes=',e,' files=',i

40
ais misqrt Management Information Systems Quarterly fixes= 0  files= 15
